# Display Rank

In MPI, each parallel process is called a *Rank*. Each rank can run in parallel. The number of ranks is configured when the Julia program is executed. 

We'll go through a small program to explain how this works. 

In [1]:
# This cell should add the packages you need for 
# this notebook (and other notebooks in this directory)
# You should get a message saying 
# "Activating project at `.../TryMPI/notebooks` where the ... is the 
# absolute path to the directory of this repository and these notebooks.
using Pkg
Pkg.activate(".")
Pkg.instantiate()

# This cell will appear in other notebooks as well

  Activating project at `~/Development/HighVelocityJuliaAnalysis/TryMPI/notebooks`


MPI programs have to be executed with a special command. For running on a Desktop or Laptop, the `MPI.jl` package provides such a command called `mpiexecjl`. We need to install this command on your machine. You only need to do this once on your laptop. Execute the cell above and the cell below to do that.

In [4]:
using MPI
MPI.install_mpiexecjl(force=true)
# This will install the mpiexecjl command to `~/.julia/bin/mpiexecjl`

┌ Info: Installing `mpiexecjl` to `/Users/lyon/.julia/bin`...
└ @ MPI /Users/lyon/.julia/packages/MPI/08SPr/src/mpiexec_wrapper.jl:21
┌ Info: Done!
└ @ MPI /Users/lyon/.julia/packages/MPI/08SPr/src/mpiexec_wrapper.jl:23


## `displayRank.jl`

We will look at `displayRank.jl` from this directory. In VSCode, display the file explorer, look in the `notebooks` directory and open `displayRank.jl`. After you open it, you can drag the tab for that file towards the bottom of the VSCode window until the blue shading is only in the lower half. That way you can see this notebook and that file at the same time. 

If within `displayRank.jl` you see `Missing reference: MPI` in orange and other errors or warnings, then you can set the Julia environment that VSCode is using. At the bottom of the VSCode window, on the status line, you'll probably see something like `Julia env: v1.7`. Click on that text. In the window that pops up, click on `(pick a folder)`. The notebook directory should appear. Click on the `Open` button. The error messages should eventually disappear. You'll note that the status line has changed.

## Running `displayRank.jl`

To run the program, open a terminal in VSCode with Cmd-\` (Command key with the backquote key). The terminal should appear at the bottom of the VSCode window. The directory should be this `notebooks` directory. Now, in the terminal, enter the following.

```bash
~/.julia/bin/mpiexecjl --project -n 4 julia displayRank.jl
```

You should see something like (the order may be different)
```
I am rank #1 of 4
I am rank #2 of 4
I am rank #3 of 4
I am rank #0 of 4
```

You may also see a window briefly appear asking if you want to accept incoming connections. It will disappear when the program ends. You can ignore this window completely. 

You can do Cmd-\` to hide the terminal window and Cmd-\` again to bring it back. 



Here's a breakdown of what `~/.julia/bin/mpiexecjl --project -n 4 julia displayRank.jl` does.

```bash
~/.julia/bin/mpiexecjl
```

runs the `mpiexecjl` program that we installed above in this notebook. This program knows how to execute in MPI and to find the Julia executable.

```bash
--project
```

tells `mpiexecjl` to use the Julia environment of our current directory. This allows Julia to find the `MPI.jl` package.

```bash
-n 4
```

tells `mpiexecjl` that we want four parallel processes. You can try more or less to see what happens (e.g. try `-n 10`).

```bash
julia displayRank.jl
```

is the program that we want to run under MPI.



## Explanation of `displayRank.jl`

The thing to remember about MPI is that it runs copies of your Julia program simulataneously - one for each process. That's why `-n 4` in the execution above gave you four lines of output. MPI ran four copies of the code simultaneously. Try `-n 10` and see what you get.

Here is an explanation of the code in `displayRank.jl`:

```julia
using MPI
MPI.Init()
```

tells Julia that you want to use the MPI package. We then need to initialize it with `MPI.Init()`. Every program using MPI must do this.

```julia
comm = MPI.COMM_WORLD
```

Each parallel process that MPI is running is a *Rank* (kind of a strange name). If you specify `-n 4` in the execution, then there will be four ranks. The ranks are numbered from 0, so `-n 4` will give Ranks with numbers 0, 1, 2, and 3. MPI allows communication between the ranks, and the way to do that is to use a *communicator*. MPI has a built in communicator called `MPI.COMM_WORLD`, that allows all of the ranks to talk to each other (the "world"). It is possible to create communicators that talk to a subset of ranks, but we won't do that yet. Because typing `MPI.COMM_WORLD` is kind of a pain, it is common to use a shorter variable name, as is done here with `comm`. 

Nearly every MPI command will need a communicator to specify the group of ranks that it should talk with. `comm` will be that communicator for us. 

```julia
myRank = MPI.Comm_rank(comm)
nRanks = MPI.Comm_size(comm)
```

These two lines get information about the Rank and how many ranks there are. Remember, MPI is running copies of this code simultaneously. Each copy has a different Rank #. Therefore, `myRank` will be different for each copy. `nRanks` will always be the same for each Rank, since the total number of ranks is a constant. Note that we use the `comm` communicator in each function call. 

```julia
print("I am rank #$(myRank) of $(nRanks)\n")
```

Print out the rank # and the number of ranks. Note that we're using Julia's string interpolation feature. `$(expression)` will insert the result of the expression into the string.

```julia
MPI.Barrier(comm)
```

Each rank can run slower or faster than the others. If you want your program to wait for all of the ranks to catch up, then use this `MPI.Barrier` function. The program won't proceed until every rank reaches this point. This is called *synchronization*. In general, synchronization makes your program slower and should be used only when necessary. We'll see examples of when we need to use it. In this case, since it is the last line of the program, we don't actually need it. There is an implicit synchronization when the program ends. 